In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import shap
from mrmr import mrmr_classif
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.linear_model import Lasso
from sklearn.feature_selection import RFE
from catboost import CatBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.decomposition import PCA
from typing import Dict
import selection_methods as smtd

In [6]:
subclasses = set()
work = [smtd.Feature_Selection]
while work:
    parent = work.pop()
    for child in parent.__subclasses__():
        if child not in subclasses:
            subclasses.add(child)
            work.append(child)
# METHODS = [x.__name__ for x in subclasses]
METHODS = list(subclasses)

In [7]:
METHODS

[selection_methods.Rf_Selection,
 selection_methods.PCA_Selection,
 selection_methods.MRMR,
 selection_methods.RFE_Selection,
 selection_methods.Lasso_Selection,
 selection_methods.Shap_Selection,
 selection_methods.GBM_Selection,
 selection_methods.Catboost_Selection,
 selection_methods.Xgb_Selection]

In [8]:
df = pd.read_csv('../data/encoded.csv')

In [9]:
df.fillna(0, inplace=True)

In [10]:
df.replace([np.inf, -np.inf], 0, inplace=True)
df.max()

gender_Male                                   1.00
SeniorCitizen_1                               1.00
Partner_Yes                                   1.00
Dependents_Yes                                1.00
PhoneService_Yes                              1.00
MultipleLines_No                              1.00
MultipleLines_No phone service                1.00
MultipleLines_Yes                             1.00
InternetService_DSL                           1.00
InternetService_Fiber optic                   1.00
InternetService_No                            1.00
OnlineSecurity_No                             1.00
OnlineSecurity_No internet service            1.00
OnlineSecurity_Yes                            1.00
OnlineBackup_No                               1.00
OnlineBackup_No internet service              1.00
OnlineBackup_Yes                              1.00
DeviceProtection_No                           1.00
DeviceProtection_No internet service          1.00
DeviceProtection_Yes           

In [11]:
data = df.drop('Churn', axis=1)
target = df['Churn']

In [12]:
importances = pd.DataFrame()

for _ in METHODS:
    method_name = _.__name__
    print(f'Method: {method_name}')
    model = _(data, target=target)
    model.fit()
    print(f'Feature importances:\n {model.get_importances()}')
    new_df = pd.DataFrame(model.get_importances(), index=[method_name])
    importances = pd.concat([importances, new_df])

Method: Rf_Selection
Feature importances:
 {'Monthly/Total_Charges': 0.12380580223378398, 'TotalCharges': 0.1227228728291543, 'TotalCharges/tenure': 0.11266508841867542, 'MonthlyCharges': 0.10890622428554292, 'Contract_Month-to-month': 0.05425600047422571, 'OnlineSecurity_No': 0.027863652807136364, 'PaymentMethod_Electronic check': 0.025814504847218433, 'InternetService_Fiber optic': 0.02537796185279866, 'TechSupport_No': 0.023887424074743504, 'gender_Male': 0.022532303539937563, 'tenure_group_0 - 11': 0.02241918399263338, 'PaperlessBilling_Yes': 0.0216184800087288, 'Partner_Yes': 0.019057908827773872, 'Contract_Two year': 0.018596059403125823, 'SeniorCitizen_1': 0.01688694055427311, 'Dependents_Yes': 0.015449944525661482, 'DeviceProtection_No': 0.014677202174384747, 'OnlineBackup_No': 0.013068338352853892, 'tenure': 0.011692109231880892, 'MultipleLines_No': 0.0111007404102539, 'OnlineBackup_Yes': 0.010727723787044873, 'PaymentMethod_Credit card (automatic)': 0.010635717545630141, 'Mul

100%|██████████| 48/48 [00:06<00:00,  7.49it/s]


Feature importances:
 {'Contract_Month-to-month': 46077019.68552643, 'PaymentMethod_Mailed check': 5965.308157861305, 'TechSupport_No': 1110.1773929484095, 'Monthly/Total_Charges': 1032.1816817053239, 'OnlineSecurity_No': 1029.7267584949716, 'PaymentMethod_Electronic check': 992.6904873312561, 'TotalCharges/tenure': 960.5523803505322, 'tenure_group_0 - 11': 840.7688953810897, 'InternetService_Fiber optic': 789.3753494014081, 'Contract_Two year': 741.9916194191692, 'OnlineBackup_No': 713.5399809592983, 'MonthlyCharges': 699.8181315325883, 'tenure': 643.7665483430388, 'DeviceProtection_No': 612.8175029387617, 'PaperlessBilling_Yes': 564.3355232628396, 'tenure_group_26 - 37': 534.8282754858046, 'Contract_One year': 520.0410786967361, 'InternetService_DSL': 502.5075407900563, 'tenure_group_65 - 76': 488.22098255343, 'SeniorCitizen_1': 465.1534954026511, 'InternetService_No': 448.7586587532255, 'Dependents_Yes': 442.66368190640566, 'OnlineSecurity_No internet service': 428.63702556069006, '

In [13]:
importances

,Monthly/Total_Charges,TotalCharges,TotalCharges/tenure,MonthlyCharges,Contract_Month-to-month,OnlineSecurity_No,PaymentMethod_Electronic check,InternetService_Fiber optic,TechSupport_No,gender_Male,...,InternetService_No,StreamingMovies_No internet service,tenure_group_65 - 76,MultipleLines_No phone service,PhoneService_Yes,DeviceProtection_No internet service,StreamingTV_No internet service,OnlineSecurity_No internet service,TechSupport_No internet service,OnlineBackup_No internet service
Rf_Selection,1.238058e-01,1.227229e-01,1.126651e-01,1.089062e-01,5.425600e-02,2.786365e-02,2.581450e-02,2.537796e-02,2.388742e-02,0.022532,...,4.892698e-03,4.018591e-03,3.502278e-03,2.893143e-03,2.809149e-03,2.796525e-03,2.380476e-03,1.800147e-03,1.426213e-03,1.122256e-03
PCA_Selection,9.111932e-33,9.111932e-33,9.111932e-33,9.111932e-33,2.046062e-10,5.022483e-08,9.111932e-33,5.445990e-08,2.699725e-08,0.999794,...,5.269234e-08,4.997204e-09,9.111932e-33,6.947010e-08,1.069592e-07,3.657477e-08,2.298690e-08,4.861269e-08,2.564714e-08,4.203763e-08
MRMR,1.032182e+03,2.960894e+02,9.605524e+02,6.998181e+02,4.607702e+07,1.029727e+03,9.926905e+02,7.893753e+02,1.110177e+03,26.063422,...,4.487587e+02,3.209240e+02,4.882210e+02,1.353051e+01,1.109210e+01,3.751804e+02,3.350048e+02,4.286370e+02,3.575197e+02,3.986753e+02
RFE_Selection,1.600000e+01,2.500000e+01,2.400000e+01,2.200000e+01,1.000000e+00,1.000000e+00,1.000000e+00,9.000000e+00,5.000000e+00,20.000000,...,1.400000e+01,1.000000e+00,1.000000e+00,1.000000e+00,1.100000e+01,7.000000e+00,1.000000e+00,1.000000e+01,1.500000e+01,1.000000e+00
Lasso_Selection,0.000000e+00,-1.083500e-04,4.218691e-03,3.821651e-03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-0.000000,...,-0.000000e+00,-0.000000e+00,0.000000e+00,0.000000e+00,-0.000000e+00,-0.000000e+00,-0.000000e+00,-0.000000e+00,-0.000000e+00,-0.000000e+00
Shap_Selection,5.242043e-01,3.966729e-01,3.711375e-01,3.178379e-01,8.569837e-01,2.682331e-01,1.745113e-01,1.936614e-01,1.906011e-01,0.068919,...,0.000000e+00,0.000000e+00,3.304113e-02,0.000000e+00,3.426257e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
GBM_Selection,1.860693e-01,4.372712e-02,4.422682e-02,4.671501e-02,3.838191e-01,6.823581e-02,3.582709e-02,8.157069e-02,4.158225e-02,0.000926,...,0.000000e+00,0.000000e+00,0.000000e+00,3.543818e-04,2.472887e-03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
Catboost_Selection,1.269020e+01,9.807683e+00,1.092686e+01,1.011001e+01,6.448553e+00,2.671594e+00,2.576723e+00,2.476236e+00,1.919217e+00,2.388539,...,1.249704e-01,5.951262e-02,3.111930e-01,4.719126e-01,2.919373e-01,2.064113e-01,5.130164e-01,2.774702e-01,2.147723e-01,1.154388e-02
Xgb_Selection,1.196427e-02,7.342538e-03,8.144485e-03,7.543160e-03,3.331451e-01,2.275703e-02,9.797708e-03,2.914353e-01,1.841668e-02,0.006710,...,0.000000e+00,0.000000e+00,4.644521e-03,0.000000e+00,1.734450e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [14]:
importances.index

Index(['Rf_Selection', 'PCA_Selection', 'MRMR', 'RFE_Selection',
       'Lasso_Selection', 'Shap_Selection', 'GBM_Selection',
       'Catboost_Selection', 'Xgb_Selection'],
      dtype='object')

In [15]:
for ind in importances.index:
    values_list = importances.loc[ind].tolist()
    non_zero_abs_values = [abs(value) for value in values_list if value != 0]
    smallest_non_zero = min(non_zero_abs_values)
    for col in importances.columns:
        if importances.loc[ind, col] == 0.0:
            importances.loc[ind, col] = smallest_non_zero / 100

In [16]:
importances

,Monthly/Total_Charges,TotalCharges,TotalCharges/tenure,MonthlyCharges,Contract_Month-to-month,OnlineSecurity_No,PaymentMethod_Electronic check,InternetService_Fiber optic,TechSupport_No,gender_Male,...,InternetService_No,StreamingMovies_No internet service,tenure_group_65 - 76,MultipleLines_No phone service,PhoneService_Yes,DeviceProtection_No internet service,StreamingTV_No internet service,OnlineSecurity_No internet service,TechSupport_No internet service,OnlineBackup_No internet service
Rf_Selection,1.238058e-01,1.227229e-01,1.126651e-01,1.089062e-01,5.425600e-02,2.786365e-02,2.581450e-02,2.537796e-02,2.388742e-02,0.022532,...,4.892698e-03,4.018591e-03,3.502278e-03,2.893143e-03,2.809149e-03,2.796525e-03,2.380476e-03,1.800147e-03,1.426213e-03,1.122256e-03
PCA_Selection,9.111932e-33,9.111932e-33,9.111932e-33,9.111932e-33,2.046062e-10,5.022483e-08,9.111932e-33,5.445990e-08,2.699725e-08,0.999794,...,5.269234e-08,4.997204e-09,9.111932e-33,6.947010e-08,1.069592e-07,3.657477e-08,2.298690e-08,4.861269e-08,2.564714e-08,4.203763e-08
MRMR,1.032182e+03,2.960894e+02,9.605524e+02,6.998181e+02,4.607702e+07,1.029727e+03,9.926905e+02,7.893753e+02,1.110177e+03,26.063422,...,4.487587e+02,3.209240e+02,4.882210e+02,1.353051e+01,1.109210e+01,3.751804e+02,3.350048e+02,4.286370e+02,3.575197e+02,3.986753e+02
RFE_Selection,1.600000e+01,2.500000e+01,2.400000e+01,2.200000e+01,1.000000e+00,1.000000e+00,1.000000e+00,9.000000e+00,5.000000e+00,20.000000,...,1.400000e+01,1.000000e+00,1.000000e+00,1.000000e+00,1.100000e+01,7.000000e+00,1.000000e+00,1.000000e+01,1.500000e+01,1.000000e+00
Lasso_Selection,1.083500e-06,-1.083500e-04,4.218691e-03,3.821651e-03,1.083500e-06,1.083500e-06,1.083500e-06,1.083500e-06,1.083500e-06,0.000001,...,1.083500e-06,1.083500e-06,1.083500e-06,1.083500e-06,1.083500e-06,1.083500e-06,1.083500e-06,1.083500e-06,1.083500e-06,1.083500e-06
Shap_Selection,5.242043e-01,3.966729e-01,3.711375e-01,3.178379e-01,8.569837e-01,2.682331e-01,1.745113e-01,1.936614e-01,1.906011e-01,0.068919,...,9.445263e-06,9.445263e-06,3.304113e-02,9.445263e-06,3.426257e-02,9.445263e-06,9.445263e-06,9.445263e-06,9.445263e-06,9.445263e-06
GBM_Selection,1.860693e-01,4.372712e-02,4.422682e-02,4.671501e-02,3.838191e-01,6.823581e-02,3.582709e-02,8.157069e-02,4.158225e-02,0.000926,...,8.779772e-07,8.779772e-07,8.779772e-07,3.543818e-04,2.472887e-03,8.779772e-07,8.779772e-07,8.779772e-07,8.779772e-07,8.779772e-07
Catboost_Selection,1.269020e+01,9.807683e+00,1.092686e+01,1.011001e+01,6.448553e+00,2.671594e+00,2.576723e+00,2.476236e+00,1.919217e+00,2.388539,...,1.249704e-01,5.951262e-02,3.111930e-01,4.719126e-01,2.919373e-01,2.064113e-01,5.130164e-01,2.774702e-01,2.147723e-01,1.154388e-02
Xgb_Selection,1.196427e-02,7.342538e-03,8.144485e-03,7.543160e-03,3.331451e-01,2.275703e-02,9.797708e-03,2.914353e-01,1.841668e-02,0.006710,...,3.734523e-05,3.734523e-05,4.644521e-03,3.734523e-05,1.734450e-02,3.734523e-05,3.734523e-05,3.734523e-05,3.734523e-05,3.734523e-05


In [17]:
zeros_count = (importances == 0).sum().sum()
zeros_count

0

In [18]:
importances.drop('PCA_Selection', axis='index')

multiplications = importances.apply(lambda x: abs(x).prod(), axis=0)

# Sort the values in descending order
multiplications_sorted = multiplications.sort_values(ascending=False)

# Create a DataFrame from the sorted values
multiplications_df = pd.DataFrame({'Column': multiplications_sorted.index, 'Multiplication': multiplications_sorted.values})

# Save the DataFrame as a CSV file
multiplications_df.to_csv('importance_values.csv', index=False)

# Print the DataFrame
print(multiplications_df)

                                     Column  Multiplication
0                   Contract_Month-to-month    3.916290e-10
1                               gender_Male    1.301266e-11
2                           SeniorCitizen_1    2.958674e-13
3               InternetService_Fiber optic    1.212830e-13
4                         OnlineSecurity_No    1.737474e-15
5                            TechSupport_No    1.086548e-15
6                            Dependents_Yes    5.538222e-17
7                           OnlineBackup_No    3.422399e-17
8                       InternetService_DSL    9.358941e-19
9                          MultipleLines_No    2.149437e-19
10                              Partner_Yes    4.392301e-20
11                         PhoneService_Yes    1.704134e-20
12                      DeviceProtection_No    1.212274e-20
13                         OnlineBackup_Yes    1.067402e-20
14                          TechSupport_Yes    6.595645e-21
15                           StreamingTV